<a href="https://colab.research.google.com/github/drmsbgr/movie-recommendation-system/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli Kütüphanelerin Yüklenmesi
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Veri Setini Colab'e İndirme (MovieLens Dataset)
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

# Veriyi Okuma
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# İlk 5 satırı görelim
print("Veri Seti Yüklendi!")
movies.head()

--2025-12-07 14:28:30--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.96.204
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  1.07MB/s    in 0.9s    

2025-12-07 14:28:32 (1.07 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  
Veri Seti Yüklendi!


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# Veri hakkında bilgi
print(f"Toplam Film Sayısı: {movies.shape[0]}")

# Genres (Türler) sütunundaki '|' işaretlerini boşlukla değiştirerek metin haline getirelim
# Bu, Scikit-learn'ün metni daha iyi işlemesini sağlar.
movies['genres'] = movies['genres'].str.replace('|', ' ')

print(movies[['title', 'genres']].head())

Toplam Film Sayısı: 9742
                                title  \
0                    Toy Story (1995)   
1                      Jumanji (1995)   
2             Grumpier Old Men (1995)   
3            Waiting to Exhale (1995)   
4  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure Animation Children Comedy Fantasy  
1                   Adventure Children Fantasy  
2                               Comedy Romance  
3                         Comedy Drama Romance  
4                                       Comedy  


In [ ]:
# TF-IDF Vektörleştiriciyi Başlatma
tfidf = TfidfVectorizer(stop_words='english')

# 'genres' sütununu matrise dönüştürme
tfidf_matrix = tfidf.fit_transform(movies['genres'])

# Matrisin boyutunu kontrol etme (Film Sayısı x Kelime Sayısı)
print(f"TF-IDF Matris Boyutu: {tfidf_matrix.shape}")

TF-IDF Matris Boyutu: (9742, 23)


In [ ]:
# Kosinüs Benzerliğini Hesaplama (Linear Kernel, büyük verilerde daha hızlıdır)
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Her filmin indeksini başlığına göre bulmak için bir harita (mapping) oluşturalım
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

In [ ]:
def film_oner(title, cosine_sim=cosine_sim):
    try:
        # 1. Filmin indeksini al
        idx = indices[title]

        # 2. Bu filmin diğer tüm filmlerle olan benzerlik puanlarını al
        sim_scores = list(enumerate(cosine_sim[idx]))

        # 3. Filmleri benzerlik puanına göre sırala (en yüksekten en düşüğe)
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 4. En benzer 10 filmi al (0. indeks kendisi olduğu için 1'den 11'e kadar)
        sim_scores = sim_scores[1:11]

        # 5. Film indekslerini al
        movie_indices = [i[0] for i in sim_scores]

        # 6. Sonuçları döndür
        return movies['title'].iloc[movie_indices]

    except KeyError:
        return "Film bulunamadı! Lütfen tam adını doğru yazdığınızdan emin olun."

# TEST EDELİM:
print("Toy Story (1995) için öneriler:")
print(film_oner('Toy Story (1995)'))

Toy Story (1995) için öneriler:
1706                                          Antz (1998)
2355                                   Toy Story 2 (1999)
2809       Adventures of Rocky and Bullwinkle, The (2000)
3000                     Emperor's New Groove, The (2000)
3568                                Monsters, Inc. (2001)
6194                                     Wild, The (2006)
6486                               Shrek the Third (2007)
6948                       Tale of Despereaux, The (2008)
7760    Asterix and the Vikings (Astérix et les Viking...
8219                                         Turbo (2013)
Name: title, dtype: object
